In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum

In [7]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

#spark = SparkSession.builder.appName("Teste").getOrCreate()



spark = SparkSession.builder \
    .appName("Demo") \
    .config("spark.hadoop.io.native.lib", "false") \
    .getOrCreate()

print("Spark está funcionando:", spark.version)


Spark está funcionando: 4.0.0


In [8]:
# Iniciar uma sessão Spark

spark = SparkSession.builder.appName("Demo").getOrCreate()

#leitura de arquivos CSV
df_vendas = spark.read.csv("vendas.csv", header=True, inferSchema=True)

#df_vendas = spark.read.csv("vendas.csv", header=True, inferSchema=True)
df_clientes =  spark.read.json("clientes.json")



In [9]:
#exibir os esquemas do dataframes
print("Esquemas do DataFrames de vendas:")
df_vendas.printSchema()

print("Esquemas do DataFrames de clientes:")
df_clientes.printSchema()

#Exibição de dados:
print("Dados de vendas:")
df_vendas.show()

print("Dados de clientes:")
df_clientes.show()     

Esquemas do DataFrames de vendas:
root
 |-- ID: integer (nullable = true)
 |-- produto: string (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- valor: double (nullable = true)

Esquemas do DataFrames de clientes:
root
 |-- Id: long (nullable = true)
 |-- _corrupt_record: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- idade: long (nullable = true)
 |-- nome: string (nullable = true)

Dados de vendas:
+---+--------------------+----------+-------+
| ID|             produto|quantidade|  valor|
+---+--------------------+----------+-------+
|  1|            Notebook|         5| 3500.5|
|  2|    Teclado Mecânico|        10|  250.9|
|  3|         Mouse Gamer|         8| 149.99|
|  4|         Monitor 27"|         4| 1299.0|
|  5|       Cadeira Gamer|         2|  899.9|
|  6|      HD Externo 1TB|         6| 320.75|
|  7|    Memória RAM 16GB|         7|  420.0|
|  8| Processador Ryzen 5|         3|  980.3|
|  9|Placa de Vídeo RT...|         1|2499.99|
| 10|  

In [10]:
#5. Filtragem

#Filtrar vendas com quantidade maior que 3
vendas_filtradas  = df_vendas.filter(col("quantidade") > 3)
print("Vendas filtradas (quantidade > 3):")
vendas_filtradas.show()

# Filtrar clientes de São Paulo
clientes_sp = df_clientes.filter(col("cidade") == "São Paulo")
print("Clientes de São Paulo:")
clientes_sp.show()


#6. agregação
# Calcular o valor total de vendas

valor_total = df_vendas.agg(sum(col("valor")).alias("valor_total"))
print("Valor total de vendas:")
valor_total.show()

#calcular a média de idade dos clientes
media_idade = df_clientes.agg(avg(col("idade")).alias("media_idade"))
print("Média de idade dos clientes:")   
media_idade.show()

# União
# Criar um novo DataFrame com informações adicionais de vendas
df_vendas_adicionais = spark.createDataFrame([
    (11, "Monitor Slim", 5, 1000.0),
    (12, "Teclado Mecanico", 2, 500.0),
    (13, "Headeset Bluetooth", 10, 200.0)
    ], ["id", "produto", "quantidade", "valor"])


#Unir os dataframens de vendas
df_vendas_uniao = df_vendas.union(df_vendas_adicionais)
df_vendas_uniao.show()

#escrita de dados
#escrever o DataFrame de vendas unificado em um arquivo CSV

#df_vendas_uniao.coalesce(1).write.csv("saida_temp", header=True, mode="overwrite")
df_vendas_uniao_pd = df_vendas_uniao.toPandas()
df_vendas_uniao_pd.to_csv("vendas_unificadas.csv", index=False)





#escrever o  datraframe de clientes em um arquivo JSON
df_clientes_pd = df_clientes.toPandas()
df_clientes_pd.to_json("clientes_exportados.json", orient="records", lines=True, force_ascii=False)



print("Dados exportados com sucesso!")  

spark.stop()  # Parar a sessão Spark



Vendas filtradas (quantidade > 3):
+---+----------------+----------+------+
| ID|         produto|quantidade| valor|
+---+----------------+----------+------+
|  1|        Notebook|         5|3500.5|
|  2|Teclado Mecânico|        10| 250.9|
|  3|     Mouse Gamer|         8|149.99|
|  4|     Monitor 27"|         4|1299.0|
|  6|  HD Externo 1TB|         6|320.75|
|  7|Memória RAM 16GB|         7| 420.0|
| 10|      Fonte 650W|         4| 379.5|
+---+----------------+----------+------+

Clientes de São Paulo:
+----+---------------+---------+-----+-------------+
|  Id|_corrupt_record|   cidade|idade|         nome|
+----+---------------+---------+-----+-------------+
|1023|           NULL|São Paulo|   34|Lucas Martins|
+----+---------------+---------+-----+-------------+

Valor total de vendas:
+-----------+
|valor_total|
+-----------+
|   10700.83|
+-----------+

Média de idade dos clientes:
+-----------+
|media_idade|
+-----------+
|       35.6|
+-----------+

+---+--------------------+----